# <center> <b> <h1> Gowalla, Stalker pattern recognition </h1></b></center>

<h4 id="challenge">Challenge</h4>

<p>Use the dataset here: <a href="https://snap.stanford.edu/data/loc-gowalla.html">https://snap.stanford.edu/data/loc-gowalla.html</a></p>

<ul>
  <li>Assume a “stalker” is someone who, in this dataset, visits some of the same locations as another person, after the other person goes to that location.</li>
  <li>A “stalker score” for a pair of people, A &amp; B, is the number of locations for which A has visited a location followed by B visiting that same location in the future.</li>
  <li>Any given location should be counted once in the score, so a stalker score can never be higher than the number of unique locations that A and B have in common.</li>
</ul>

<p>Use the datasets from the web page above to answer the following questions:</p>

<ol>
  <li>Which friend pair has the highest “stalker score”?</li>
  <li>Which non-friend pair has the highest “stalker score”?</li>
</ol>

<p>You can use any tools you want to solve this puzzle, except asking for help from other people. Please feel free to email at any time for any clarifications.</p>

<p>Please give the winning user id pairs and “stalker score” for each question, and please explain your solution methods, including any source code if you wrote any.</p>

<p>There are many libraries available that can make this challenge easier, so we are offering bonus points for:</p>
<ul>
  <li>Using as little RAM as possible</li>
  <li>Exection speed</li>
  <li>Pure python solution using base modules</li>
</ul>


# <b> <h3> <em>Script and Macine characteristics</em> </h3></b>

<p style="text-align:justify">
    This project will be implemented with PIP-8 style guide of python, I will evaluate computation time, size in memory, if it is possible, I will implement concurrency.The computer in which this scrip will be developed has a CPU intel core i5 of third generation with four threads and 2.5 Gz, 8GB of RAM, standard hard-disk of 512GB,  executing Ubuntu 16.04.

This script is powered by python 3.7 (anaconda distribution). 
</p>

# 1. Loading Datasets

In [1]:
import pandas as pd 

PATH_CHECKINGS = "datasets/Gowalla_totalCheckins.txt"
PATH_EDGES = "datasets/Gowalla_edges.txt"

In [2]:
data_edges = pd.read_csv(PATH_EDGES, sep = "\t", header = None)
data_edges.columns = ["user", "friend"]

In [3]:
data_edges.head()

,user,friend
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5


In [4]:
data_checkins = pd.read_csv(PATH_CHECKINGS, sep="\t", header=None)
data_checkins.columns = ["user", "check-in time", "latitude", "longitude", "location id"]
data_checkins.set_index("location id", inplace = True)

In [5]:
data_checkins.head(5)

,user,check-in time,latitude,longitude
location id,,,,
22847,0,2010-10-19T23:55:27Z,30.235909,-97.795140
420315,0,2010-10-18T22:17:43Z,30.269103,-97.749395
316637,0,2010-10-17T23:42:03Z,30.255731,-97.763386
16516,0,2010-10-17T19:26:05Z,30.263418,-97.757597
5535878,0,2010-10-16T18:50:42Z,30.274292,-97.740523


The plan to addres this problem wil be:

-  <b> <em> Preprocess the raw-data: </em></b></i> <p style="text-align:justify">let's deletete the information that there is not important for our challenge, (<b><em>latitude, longitude</em></b>). in each step it will shown how change the size of the data. I will create a Sparse matrix Object to represent the matrices. we will have 3 matrix: friend_relationship, stalker_score, locations_chekins. for better explanation, in each step there will be a little discription about what I'm doing.</p>


# 2. Preprocessing Data

## 2.1 Cleaning of  "latitude" and "longitude" 

In [6]:
import sys

def size_mb(x):
    """return the size in megabytes of one obeject"""
    
    bytes_size = sys.getsizeof(x)
    
    return (bytes_size /(1024*1024))


In [7]:
print("Size of checkins raw_dataset: %.2f MB" %  size_mb(data_checkins))

Size of checkins raw_dataset: 669.74 MB


In [8]:
print("Size of Friendship network raw_dataset: %.2f MB" %  size_mb(data_edges))

Size of Friendship network raw_dataset: 29.00 MB


In [9]:
#Elimination of latitude and longitude columns
data_checkins.drop(['latitude', 'longitude'], axis=1, inplace = True)

In [10]:
print("Size of checkins without latitude and longitude: %.2f MB" %  size_mb(data_checkins))

Size of checkins without latitude and longitude: 571.43 MB


## 2.2 Pre-processing check-in time
Now the date is type string, let's change it to python date type.

In [11]:
data_checkins['check-in time'] = pd.to_datetime(data_checkins['check-in time'], infer_datetime_format=True)

In [12]:
data_checkins.sort_values(by=['location id'], inplace = True)
data_checkins.head()

,user,check-in time
location id,,
8904,343,2009-08-21 03:01:19
8904,576,2009-10-05 20:19:24
8904,24,2009-02-05 06:27:43
8904,310,2009-12-06 22:51:49
8904,5164,2010-01-25 01:29:46


In [13]:
print("Size of checkins with date pre-processed: %.2f MB" %  size_mb(data_checkins))

Size of checkins with date pre-processed: 147.47 MB


<p style="text-align:justify"> We could see how just adjusting the datatype of the <b><em>"check-in time"</em></b> we achieve a huge reduction of the size of our principal dataset.

Let's check if there is some missing value
</p>

## 2.3 Checking missing values

In [14]:
data_checkins.isna().sum()

user             0
check-in time    0
dtype: int64

In [15]:
data_edges.isna().sum()

user      0
friend    0
dtype: int64

<p> There is not missing values in both datasets, so we can continue.

I think we can reduce even more the datasets size.
</p>

## 2.4 sparse matrix

<p style="text-align:justify"> To solve our challenge we can implemnt sparce matrices, with help of dicts.</p> 
     

In [62]:
class SparceMatrix:
    """ 
    
    Object for manage big matrices with a 
    hugue amount of NaN or repteaded values.
    
    based in python dicts and list, 
    list just for unidimensionals
    """
    
    def __init__(self):
        self.__matrix = {} #private attribute
    
    def get_row(self,row_id):
        if row_id in self.__matrix:
            return self.__matrix[row_id]
        return False;
    
    def element_exist(self, row_id, col_id):
        try:
            if not row_id in self.__matrix:
                return False

            if col_id in self.get_row(row_id):
                return True
            return False
        except ValueError:
            print("you are using a unidimesional version, this version not has col_id") 
    
    def get_val(self, row_id, col_id):
        try:
            return self.__matrix[row_id][col_id]
        except ValueError:
            print("you are using a unidimesional version, this version not has col_id") 
        #return self.__matrix[row_id][col_id]
    
    def get_index(self):
        return self.__matrix.keys()
    
    def get_matrix(self):
        return self.__matrix
    
    def append(self, k, row_id, col_id = None):
        """
        add a new item.
        
        k could be any type of data.
        """
        if col_id is None:
            if  row_id in self.__matrix:
                self.__matrix[row_id].append(k)
            else:
                self.__matrix[row_id] = [k]
        else:
            if  row_id in self.__matrix:
                self.__matrix[row_id].update({col_id: k})
            else:
                self.__matrix[row_id] = {col_id: k}
       
    
    def update(self, k, row_id, col_id = None):
        """
        update item in the position [row_id, col_id].
        
        k could be any type of data.
        """
        
        if col_id is None:
            self.__matrix[row_id] = k
        else:
            self.__matrix[row_id][col_id] = k 
        
        
    def concat(self, Matrix):
        self.__matrix.update(Matrix.get_matrix())
    
    def pprint(self, k = 4):
        """ 
        print the rows of the matrix
        
        by defect print the first 5 
        """
        
        counter = 0
        for x, y in self.__matrix.items():
            print(f'{x} -> {y}')
            counter += 1
            if counter == k :
                break  

# 2.5 transforming representation of data

<p style="text-align:justify"> Now, that we have our class SparceMatrix we can represent our datasets: Checkins and friendship network, and our result set: stalker_classification, in this data structure. It definetly will reduce the size in RAM of our datasets and it will provide more performance in our <b><em>step 3. Processing.</em></b></p>

### 2.5.1 Transformation of data_checkins

<p> For this transformation let's make some asumptions:
    <ul>
        <li>based on the definition of the challenger: <em>"stalker is someone who, in this dataset, visits some of the same locations as another person, after the other person goes to that location."</em>  we could say that there is not timestap defined, in this way we could say that if the person <em>A</em> went to the place <em>X</em> the 01/01/2020 and the person <em>B</em> went to the same place the 05/01/2020 B will be considered has staleker of <em>A</em>. It means that while: $$A_{date}(X) < B_{date}(X) \implies (A,B) = 1 $$
this is valid for each place</li>
        <li>
            Using the previous assumption we could say that for our analysis we only need at most two dates for each user over one place: the first date that the user has visted the place and the last date, obviously there are users that has visited a certain place just one time, in this case we will use just one date. It will reduce the size of our dataset, because there are user that are made chekin more than two times in the same place, for example in their jobs. 
        </li>
        <br>
        <li>
            if A place x just has been visited by one user, for our challenge is useless, so let delete this places, it will reduce more the size of our pre-procesed dataset
        </li>
    </ul>
</p>

In [17]:
# we will use this to measure the time that takes execute a function
import time

In [18]:
#first create our sparse matrix 
locations_chekins = SparceMatrix()

In [19]:
#lets extract the unique ids of the places
location_ids = data_checkins.index.unique()
print(len(location_ids))

1280969


In [22]:
#lets calc the time:
# Start the stopwatch / counter 
start = time.time()
for location_id in location_ids:
    #select the users and dates
    location_checkins = data_checkins.loc[location_id]
    #ommiting the places with just one visitor
    if location_checkins.size > 2:
        #lets change the index for the users id
        location_checkins.reset_index(inplace = True)
        location_checkins.set_index("user", inplace = True)
        #lets extract the unique ids of the users that visited the place
        user_ids = location_checkins.index.unique()
        #ommit locations with just one visitor
        if len(user_ids) > 1:
            #lets extract the dates in wich each user has visited the place
            for user_id in user_ids:
                user_dates = location_checkins.loc[user_id]
                #more than one visit?
                # two because a df with just one register has a dimensionality of 2 
                if user_dates.size > 2:
                    #extract the first and last date
                    first_date = user_dates["check-in time"].min()
                    last_date = user_dates["check-in time"].max()
                    #lets add this register to our SparceMatrix
                    locations_chekins.append([first_date,last_date], location_id, user_id)
                else:
                    date = user_dates["check-in time"]
                    #lets add this register to our SparceMatrix
                    locations_chekins.append(date, location_id, user_id)
        else:
            pass
    else:
        pass
            
#cleaning vars
del location_id, location_checkins, user_ids, user_id, user_dates, first_date, last_date, date

# Stop the stopwatch / counter 
stop = time.time() 
print("This is the time that have taked pre process the data_checkins")
print(stop - start)

This is the time that have taked pre process the data_checkins
2685.626620054245


In [20]:
import pickle

In [21]:
#load / save the pre-processed files
def load_obj(path):
    pickle_in = open(path,"rb")
    obj = pickle.load(pickle_in)
    return obj

def save_obj(obj, path):
    #save the preprocessed data
    pickle_out = open( path + ".pickle","wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

In [22]:
import gc
gc.collect()

120

In [22]:
# save_obj(locations_chekins, "locations_chekins")
locations_chekins = load_obj("locations_chekins.pickle")

#### lets see the size of our dataset after the preprocessing

In [23]:
print("this is the size of our dataset already pre-processed: ")
print(size_mb(locations_chekins.get_matrix()))

this is the size of our dataset already pre-processed: 
40.00010681152344


<p> we could see that from our initial size of 700 MB we have reduce it until 40 MB it is a reduction of 94.29%. For other hand the pre process has take arround 43 minutes, let see if we could implement concurrency to reduce this time and get the same result</p>

#### 2.5.1.1 applying concurrency

In [23]:
import concurrent.futures
import math

In [24]:
# we will use this  function for the concurrency
def split_list(array, n_splits = 4):
    """ this function split a lsit in almost equal parts"""
    
    splits = []
    delta = math.floor((len(array)/n_splits))
    for i in range(n_splits):
        if i == n_splits - 1 :
            a = array[(i*delta): ]
            splits.append(a)
        else:
            a = array[(i*delta) : ((i+1)*delta)]
            splits.append(a)
            
    return splits

In [25]:
def location_preprocessing(loc_ids, data_checkins):
    """ 
    this function will be executed for the threads
    
    this function will extract our useful data 
    of a user that has visted a place 
    """
    
    temp = SparceMatrix()
    
    for location_id in loc_ids:
        #select the users and dates
        location_checkins = data_checkins.loc[location_id]
        #ommiting the places with just one visit
        if location_checkins.size > 2:
            #lets change the index for the users id
            location_checkins.reset_index(inplace = True)
            location_checkins.set_index("user", inplace = True)
            #lets extract the unique ids of the users that visited the place
            user_ids = location_checkins.index.unique()
            #ommit locations with just one visitor
            if len(user_ids) > 1:
                #lets extract the dates in wich each user has visited the place
                for user_id in user_ids:
                    user_dates = location_checkins.loc[user_id]
                    #more than one visit?
                    # two because a df with just one register has a dimensionality of 2 
                    if user_dates.size > 2:
                        #extract the first and last date
                        first_date = user_dates["check-in time"].min()
                        last_date = user_dates["check-in time"].max()
                        #lets add this register to our SparceMatrix
                        temp.append([first_date,last_date], location_id, user_id)
                    else:
                        date = user_dates["check-in time"]
                        #lets add this register to our SparceMatrix
                        temp.append(date, location_id, user_id)
            else:
                pass
        else:
            pass

    #cleaning vars
    del location_id, location_checkins, user_ids, user_id, user_dates, first_date, last_date, date
    return temp
    
    

In [70]:
#lets create te sparce matrix for concurrency
concurrency_locations_chekins = SparceMatrix()

In [27]:
# Start the stopwatch / counter 
start = time.time()

with concurrent.futures.ProcessPoolExecutor() as executor:
    split_loc_ids = split_list(location_ids, 6)
    results = [executor.submit(location_preprocessing, split, data_checkins) for split in split_loc_ids]
    
    for f in concurrent.futures.as_completed(results):
        concurrency_locations_chekins.concat(f.result())
        
# Stop the stopwatch / counter 
stop = time.time() 
print("This is the time that have taked pre process concurrently the data_checkins")
print(stop - start)

This is the time that have taked pre process concurrently the data_checkins
1326.3602726459503


In [73]:
#save_obj(concurrency_locations_chekins, "output/concurrency_locations_chekins")
concurrency_locations_chekins = load_obj("output/concurrency_locations_chekins.pickle")

In [74]:
print("this is the size of our dataset already pre-processed: ")
print(size_mb(concurrency_locations_chekins.get_matrix()))

this is the size of our dataset already pre-processed: 
20.000099182128906


In [37]:
# we dont need more this raw dataset
del data_checkins

### We see that we could have the same result in almost te middle of the time, i think that whit a graphic card it could be even faster

### 2.5.2 transforming  data_edges
<p> Like the data_checkins this information will be stored in a sparse matrix. We could say that, for be a non directed graph,  if $u_0$ is friend of $u_1$ so $u_1$ is friend of $u_0$. $$ if (u_0,u_1) \implies (u_1,u_0)$$ In this way and for use less space and perform the search fastly we could sort ascendig the user_id, and store the friendship relation  between ($u_0$, $u_1$) in the user list that has the smallest id</p>

In [30]:
data_edges.set_index("user", inplace=True)
#sort the values is important for the concurrency that we will apply
data_edges.sort_index(inplace=True)
data_edges.head()

,friend
user,
0,1
0,2
0,3
0,4
0,5


In [31]:
print("Size of raw data_edges: %.2f MB" %  size_mb(data_edges))

Size of raw data_edges: 29.00 MB


In [32]:
#let see the numebr of unique users
users = data_edges.index.unique()

In [33]:
print(len(users))

196591


In [34]:
def edges_preprocessing(user_ids, data_edges):
    """
    this function will be used for concurrency
    
    it will return a sparece matrix with the 
    friend list of a certain number of users. 
    """
    temp = SparceMatrix()
    for user_id in user_ids:
        #friend of the user_id
        user_friends = data_edges.loc[user_id]
        #list of frieds
        if user_friends.size > 1:
            friend_list = user_friends["friend"].values

            for friend in friend_list:
                # we will just store te friendship of te user that
                # have an id bigert than the user_id
                if user_id < friend:
                    # friend = value, userd_id = row_id
                    temp.append(friend, user_id)
        else:
            unique_friend = user_friends.friend
            if user_id < unique_friend:
                temp.append(unique_friend, user_id)
        
    return temp
        

In [79]:
# this matrix will be unidimensional
concurrency_friendship = SparceMatrix()

In [36]:
# Start the stopwatch / counter 
start = time.time()

with concurrent.futures.ProcessPoolExecutor() as executor:
    split_user_ids = split_list(users, 6)
    results = [executor.submit(edges_preprocessing, split, data_edges) for split in split_user_ids]
    
    for f in concurrent.futures.as_completed(results):
        concurrency_friendship.concat(f.result())
        
# Stop the stopwatch / counter 
stop = time.time() 
print("This is the time that have taked pre process concurrently the data_edges")
print(stop - start)

This is the time that have taked pre process concurrently the data_checkins
37.947980642318726


In [83]:
#save_obj(concurrency_friendship, "output/concurrency_friendship")
concurrency_friendship = load_obj("output/concurrency_friendship.pickle")

In [39]:
print("this is the size of our dataset already pre-processed: ")
print(size_mb(concurrency_friendship.get_matrix()))

this is the size of our dataset already pre-processed: 
5.000099182128906


In [42]:
# we don't need more the dataset 
del data_edges

 ### Almost 84% of size reduction and processed in just  36 seconds

# 3. Processing the Challenge

## 3.1 Stalker score

<p>let's remeber:</p>

<ul>
  <li>A “stalker score” for a pair of people, A &amp; B, is the number of locations for which A has visited a location followed by B visiting that same location in the future.</li>
  <li>Any given location should be counted once in the score, so a stalker score can never be higher than the number of unique locations that A and B have in common.</li>
</ul>

<p>
    We will represent the result as A Sparce Matrix, the row_id will be the user that has been stalked A), the col_id is te user that stalks (B), and the value is in how many locations is the same. 
</p>

In [100]:
stalker_score = SparceMatrix()

In [103]:
# Start the stopwatch / counter 
start = time.time()

for location_id in concurrency_locations_chekins.get_index():
    users_location_checkins = concurrency_locations_chekins.get_row(location_id)
    #for x, y in self.__matrix.items():
    for user, dates in users_location_checkins.items():
        if type(dates) == list:
            current_first = dates[0]
            current_last = dates[1]
        else:
            current_first = dates
            current_last = dates
            
        for next_user, next_user_dates in users_location_checkins.items():
            if user != next_user:
                if type(next_user_dates) == list:
                    next_first = next_user_dates[0]
                    next_last = next_user_dates[1]
                else:
                    next_first = next_user_dates
                    next_last = next_user_dates
                
                # double way comparison 
                # if the first date of the current user is before the lastone date of the next user
                # next is stalking current
                # has to be just smallest comparison because the challene say in the future
                if current_first < next_last:
                    if stalker_score.element_exist(user, next_user):
                        score = stalker_score.get_val(user, next_user) 
                        stalker_score.update(score + 1, user, next_user)
                    else:
                        stalker_score.append(1, user, next_user)
                # if the first date of next user is before the lastone date of the curren user
                # current user is stalking the nextuser
                if next_first < current_last:
                    if stalker_score.element_exist(next_user, user):
                        score = stalker_score.get_val(next_user, user) 
                        stalker_score.update(score + 1, next_user, user)
                    else:
                        stalker_score.append(1, next_user, user)
                else:
                    # is not a stalker
                    pass
            else:
                # you can't stalk yourself
                pass
            
# Stop the stopwatch / counter 
stop = time.time() 
print("This is the time that have taked pre process the stalker score")
print(stop - start)

KeyboardInterrupt: 

In [ ]:
for location in concurrency_locations_chekins.get_index():
    


for user, dates in k.items():
    current_first = dates[0]
    current_last = dates[1]
    for next_user, next_dates in k.items():
        if user != next_user:
            next_first = next_dates[0]
            next_last = next_dates[1]
            
            # comparacion de doble via
            
            # si la primera date del current es menor a la ultima date de check-in del next
            # next es saltalker de  current
            # estrictamente menor porque  el reto dice explicitamente en el futuro.
            if current_first < next_last:
                if m2.exist(user, next_user):
                    score = m2.get_val(user, next_user) 
                    m2.update(user, next_user, score + 1)
                else:
                    m2.append(user, next_user, 1)
            # si la primera data the next user es menor a la ultima data de current user
            #current user es stalker the next
            if next_first < current_last:
                if m2.exist(next_user, user):
                    score = m2.get_val(next_user, user) 
                    m2.update(next_user, user, score + 1)
                else:
                    m2.append(next_user, user, 1)
                
            else:
                pass
        else:
            pass
        
        
    

In [102]:
k = (sorted(concurrency_friendship.get_matrix().items()))

In [105]:
k.get_index()

AttributeError: 'list' object has no attribute 'get_index'

In [90]:
concurrency_friendship.get_index()

dict_keys([147441, 147443, 147444, 147445, 147446, 147448, 147450, 147454, 147456, 147457, 147459, 147460, 147465, 147467, 147468, 147471, 147472, 147473, 147474, 147478, 147481, 147482, 147483, 147484, 147486, 147487, 147490, 147495, 147496, 147497, 147498, 147499, 147501, 147502, 147505, 147506, 147507, 147508, 147509, 147511, 147514, 147523, 147539, 147540, 147541, 147543, 147544, 147545, 147547, 147548, 147549, 147550, 147552, 147553, 147554, 147557, 147558, 147561, 147567, 147575, 147576, 147578, 147580, 147581, 147582, 147584, 147585, 147586, 147588, 147589, 147590, 147591, 147592, 147593, 147594, 147595, 147596, 147597, 147598, 147599, 147601, 147602, 147604, 147605, 147608, 147609, 147612, 147614, 147616, 147617, 147618, 147619, 147620, 147621, 147624, 147625, 147626, 147627, 147630, 147633, 147634, 147635, 147636, 147639, 147642, 147643, 147644, 147645, 147647, 147648, 147650, 147655, 147657, 147658, 147659, 147661, 147662, 147664, 147665, 147670, 147671, 147679, 147681, 14768

In [70]:
k = data_edges.loc[310]

In [72]:
k.size

1

In [74]:
type(k.friend)

numpy.int64

In [56]:
k["friend"]

0

In [ ]:
import random
aleatorios = [random.randint(0,1000) for _ in range(100)]

In [ ]:
data_checkins.index.unique()

In [143]:
k = data_checkins.loc[8904]
k.shape

(12, 2)

In [144]:
k.reset_index(inplace=True)
k

,location id,user,check-in time
0,8904,343,2009-08-21 03:01:19
1,8904,576,2009-10-05 20:19:24
2,8904,24,2009-02-05 06:27:43
3,8904,310,2009-12-06 22:51:49
4,8904,5164,2010-01-25 01:29:46
5,8904,256,2010-09-19 01:47:05
6,8904,343,2009-05-28 23:40:31
7,8904,392,2009-12-17 23:02:42
8,8904,256,2010-04-24 23:57:00
9,8904,343,2009-06-20 01:40:01


In [145]:
k.set_index("user", inplace = True)

In [146]:
len(k.index.unique())

7

In [100]:
h = k.loc[44458]
print(h["check-in time"])

user
44458   2010-07-25 18:13:48
44458   2010-04-20 17:56:37
Name: check-in time, dtype: datetime64[ns]


In [101]:
print(h.size)

4


In [78]:
h = k.loc[44458]
print(h["check-in time"].min())
print(h["check-in time"].max())

2010-04-20 17:56:37
2010-07-25 18:13:48


In [89]:
if not hsize:
    print("eureka")

IndexError: tuple index out of range

In [ ]:
data_checkins

In [ ]:
k = data_checkins.iloc[0]
for i in k:
    print(i)

In [ ]:
#let's transform our dataset
x = 4
k = data_checkins.iloc[:x]
print(k)

In [63]:
m = SparceMatrix()
m.append([1,10], 1, 1)
m.append([3,10], 1, 2)
m.append([1,10], 1, 3)
m.append([3,10], 1, 4)



In [65]:
print(m.get_row(6))

False


In [ ]:
k = m.get_row(1)

In [ ]:
print(k)

In [ ]:
m.exist(123, 4)

In [ ]:
m2 = SparceMatrix()

In [ ]:
for user, dates in k.items():
    current_first = dates[0]
    current_last = dates[1]
    for next_user, next_dates in k.items():
        if user != next_user:
            next_first = next_dates[0]
            next_last = next_dates[1]
            
            # comparacion de doble via
            
            # si la primera date del current es menor a la ultima date de check-in del next
            # next es saltalker de  current
            # estrictamente menor porque  el reto dice explicitamente en el futuro.
            if current_first < next_last:
                if m2.exist(user, next_user):
                    score = m2.get_val(user, next_user) 
                    m2.update(user, next_user, score + 1)
                else:
                    m2.append(user, next_user, 1)
            # si la primera data the next user es menor a la ultima data de current user
            #current user es stalker the next
            if next_first < current_last:
                if m2.exist(next_user, user):
                    score = m2.get_val(next_user, user) 
                    m2.update(next_user, user, score + 1)
                else:
                    m2.append(next_user, user, 1)
                
            else:
                pass
        else:
            pass
        
        
    

In [ ]:
m2.pprint()

<p> After create our class SparseMatrix we will start to transform our datasets in 
     The matrixes that we'll implement are shown in the next images:

In [ ]:
b = { 
    1:[1212, 1414],
    2:[1313, 1515]
    }

In [ ]:
a = {
    123 : { 
        1:[1212, 1414],
        2:[1313, 1515]
    },
    234 : [{1:[1212, 1414]}, {2:[1313, 1515]}]
}

In [ ]:
a[123][1]